In [3]:
# Installing required packages
!pip install yfinance pandas numpy matplotlib

# Importing required libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 7 NYSE traded stocks and the ETFs
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'JPM', 'V']
etfs = ['SPY', 'IWM', 'DIA']

# Historical data for stocks and ETFs (10 years)
data = yf.download(tickers + etfs, start='2014-09-30', end='2024-09-30')['Adj Close']

# Calculating portfolio weights (Equal weight)
weights = np.array([1/len(tickers)] * len(tickers))

# Calculating returns (daily returns)
returns = data.pct_change().dropna()

# Calculating trailing 3-month volatility (Annualized Volatility)
volatility_3m = returns.rolling(window=63).std() * np.sqrt(252)

# Annualizing Volatility for each stock (Last 3 months)
volatility_annualized = volatility_3m.iloc[-1][tickers]

[*********************100%***********************]  10 of 10 completed


In [5]:
# Calculating beta (Trailing 12-months)
def calculate_beta(stock_returns, market_returns):
    covariance = np.cov(stock_returns, market_returns)[0, 1]
    market_variance = np.var(market_returns)
    return covariance / market_variance

In [6]:
# Beta against SPY, IWM, and DIA (Trailing 12-months)
beta_spy = []
beta_iwm = []
beta_dia = []
for stock in tickers:
    beta_spy.append(calculate_beta(returns[stock][-252:], returns['SPY'][-252:]))
    beta_iwm.append(calculate_beta(returns[stock][-252:], returns['IWM'][-252:]))
    beta_dia.append(calculate_beta(returns[stock][-252:], returns['DIA'][-252:]))

# Calculating drawdowns (52-week High-Low drawdown)
high_52week = data.rolling(window=252).max()
low_52week = data.rolling(window=252).min()
drawdown_avg = (high_52week - low_52week) / high_52week
drawdown_max = (data.max() - data.min()) / data.max()

In [7]:
# Calculating total return and annualized total return (Trailing 10-years)
total_return = data.iloc[-1] / data.iloc[0] - 1
annualized_return = (1 + total_return) ** (1 / 10) - 1

In [8]:
# Table showing the portfolio risk analysis for the 7 stocks
portfolio_analysis = pd.DataFrame({
    'Ticker': tickers,
    'Portfolio Weight': weights,
    'Annualized Volatility (3M)': volatility_annualized.values,
    'Beta vs SPY': beta_spy,
    'Beta vs IWM': beta_iwm,
    'Beta vs DIA': beta_dia,
    'Avg Weekly Drawdown': drawdown_avg.iloc[-1][tickers].values,
    'Max Weekly Drawdown': drawdown_max[tickers].values,
    'Total Return (10Y)': total_return[tickers].values,
    'Annualized Total Return (10Y)': annualized_return[tickers].values
})

In [9]:
# Display the portfolio risk analysis
portfolio_analysis

,Ticker,Portfolio Weight,Annualized Volatility (3M),Beta vs SPY,Beta vs IWM,Beta vs DIA,Avg Weekly Drawdown,Max Weekly Drawdown,Total Return (10Y),Annualized Total Return (10Y)
0,AAPL,0.142857,0.231065,0.996152,0.327179,0.801046,0.298286,0.911660,9.185492,0.261241
1,MSFT,0.142857,0.205821,1.101404,0.255306,0.931630,0.333528,0.925521,9.798036,0.268629
2,GOOGL,0.142857,0.263921,1.298203,0.367396,0.896363,0.361701,0.870151,4.586444,0.187719
3,AMZN,0.142857,0.324631,1.507641,0.488227,1.183459,0.402150,0.928263,10.659224,0.278401
4,TSLA,0.142857,0.662447,2.215996,1.020424,1.808858,0.461156,0.976637,15.098977,0.320322
5,JPM,0.142857,0.237270,0.654511,0.399869,1.071740,0.406862,0.814179,3.609103,0.165096
6,V,0.142857,0.215743,0.622381,0.212150,0.804384,0.221352,0.839904,4.532643,0.186570


In [10]:
# Calculate portfolio returns
portfolio_returns = (returns[tickers] * weights).sum(axis=1)

In [11]:
# Calculating Portfolio Risk against the ETFs (SPY, IWM, DIA)
portfolio_returns = (returns[tickers] * weights).sum(axis=1)

# Calculating correlations between portfolio and ETFs
correlations = returns[etfs].corrwith(portfolio_returns)

# Calculating covariance between portfolio returns and each ETF
covariances = {}
for etf in etfs:
    covariances[etf] = np.cov(portfolio_returns, returns[etf])[0, 1]

# Converting covariances to a list
covariances_values = list(covariances.values())

# Calculating tracking error (standard deviation of the difference in returns)
tracking_errors = []
for etf in etfs:
    tracking_error = np.std(portfolio_returns - returns[etf]) * np.sqrt(252)
    tracking_errors.append(tracking_error)

In [12]:
# Assuming a risk-free rate of 4% for Sharpe Ratio calculation
risk_free_rate = 0.04
portfolio_mean_return = portfolio_returns.mean() * 252
portfolio_volatility = portfolio_returns.std() * np.sqrt(252)
sharpe_ratios = []
for etf in etfs:
    etf_return = returns[etf].mean() * 252
    etf_volatility = returns[etf].std() * np.sqrt(252)
    sharpe_ratio = (etf_return - risk_free_rate) / etf_volatility
    sharpe_ratios.append(sharpe_ratio)

# Calculating Volatility Spread
etf_volatility = returns[etfs].std() * np.sqrt(252)
volatility_spread = portfolio_volatility - etf_volatility

# Create the portfolio_vs_etf DataFrame
portfolio_vs_etf = pd.DataFrame({
    'ETF Ticker': etfs,
    'Correlation with Portfolio': correlations.values,
    'Covariance': covariances_values,  # Covariance between portfolio and ETFs
    'Tracking Error': tracking_errors,
    'Sharpe Ratio': sharpe_ratios,
    'Volatility Spread': volatility_spread.values
})

# Display the portfolio risk vs ETF analysis
portfolio_vs_etf


,ETF Ticker,Correlation with Portfolio,Covariance,Tracking Error,Sharpe Ratio,Volatility Spread
0,SPY,0.891872,0.000149,0.113609,0.566831,0.061792
1,IWM,0.743331,0.000158,0.166224,0.307170,0.013860
2,DIA,0.808298,0.000133,0.141402,0.501302,0.063603


In [13]:
# Creating a correlation matrix between portfolio, ETFs, and individual stocks
all_assets = tickers + etfs
correlation_matrix = returns[all_assets].corr()

# Display the correlation matrix
print("\nCorrelation Matrix:")
print(correlation_matrix)


Correlation Matrix:
Ticker      AAPL      MSFT     GOOGL      AMZN      TSLA       JPM         V  \
Ticker                                                                         
AAPL    1.000000  0.681530  0.606787  0.554666  0.429256  0.425805  0.564993   
MSFT    0.681530  1.000000  0.715195  0.648939  0.406513  0.448641  0.629530   
GOOGL   0.606787  0.715195  1.000000  0.644729  0.367187  0.411741  0.559813   
AMZN    0.554666  0.648939  0.644729  1.000000  0.391749  0.291405  0.457738   
TSLA    0.429256  0.406513  0.367187  0.391749  1.000000  0.262125  0.340058   
JPM     0.425805  0.448641  0.411741  0.291405  0.262125  1.000000  0.578935   
V       0.564993  0.629530  0.559813  0.457738  0.340058  0.578935  1.000000   
SPY     0.749122  0.795194  0.722186  0.621783  0.469929  0.726167  0.766182   
IWM     0.584249  0.588865  0.561019  0.475981  0.445725  0.705375  0.635865   
DIA     0.676620  0.712868  0.621704  0.499697  0.389479  0.788036  0.768270   

Ticker       SPY  